In [1]:
cd ..

/Users/gyanarora/Desktop/Work/GenAI/lmflow


In [2]:
import os
import json
from lmflow.lm import GroqChatLM
from lmflow.base import lmtool, ChatLM

key = json.load(open("rough/keys.json"))['groq']

In [3]:
import pathlib
from datetime import datetime

@lmtool()
def get_current_time():
    '''Returns the current date and time of the local system'''
    return datetime.now().strftime("Date: %m-%d-%Y, Time: %H:%M:%S")

@lmtool()
def get_day(date):
    '''Returns the calendar day at the given date
    param: date (str)
    '''
    map_dict = {0: 'Monday', 1:'Tuesday', 2:'Wednesday', 3:'Thursday', 4:'Friday', 5:'Saturday', 6:'Sunday'}
    return f'{date} was a ' + map_dict[datetime.strptime('11-24-2024', '%m-%d-%Y').weekday()]

@lmtool()
def get_file_size(file_path):
    '''Returns the size of the file at the given path

     # Parameters:
    file_path (str): path of the file

    # Returns:
    size (int): size of the file at file_path in bytes
    '''
    return f'The size of the file at path {file_path} is {os.path.getsize(file_path)} bytes'

@lmtool(human=True, dont_infer=True)
def delete_file(file_path):
    '''deletes the file at the given file path
    param: file_path
    '''
    os.remove(file_path)
    return f'Deleted file {file_path}'


@lmtool()
def list_files():
    '''lists files in current directory'''
    return '\n'.join(os.listdir())


@lmtool(dont_infer=True)
def login():
    '''Logs in to the file system to enable file operations'''
    return 'Login Successfull'

@lmtool(dont_infer=True)
def change_directory(dir):
    '''changes directory to the given path
    param: dir
    '''
    os.chdir(dir)
    return 'Changed to ' + dir

@lmtool()
def get_current_directory():
    '''Returns the path to current directory'''
    return os.getcwd()

In [4]:
local_timekeeper = GroqChatLM(key, model='llama3-70b-8192', max_tokens=1024, temperature=0.1, name='timekeeper')
local_timekeeper.set_specialisation("This agent specialises in information about current date and time")
local_timekeeper.add_tools([get_current_time, get_day])

file_operator_agent = GroqChatLM(key, model='llama3-70b-8192', max_tokens=1024, temperature=0.1, name='file_operator_agent')
file_operator_agent.set_specialisation("This agent specialises in all modifying and getting details about infividual files")
file_operator_agent.add_tools([get_file_size, delete_file])

file_system_agent = GroqChatLM(key, model='llama3-70b-8192', max_tokens=1024,temperature=0.1, name='file_explorer_agent')
file_system_agent.set_specialisation("This agent specialises in exploring and navigating local directory")
file_system_agent.add_tools([list_files, change_directory, get_current_directory])
file_system_agent.add_agents(file_operator_agent)

supervisor = GroqChatLM(key, model='llama3-70b-8192', max_tokens=1024, temperature=0.1, name='supervisor')
supervisor.add_tools(login)
supervisor.add_agents([local_timekeeper, file_system_agent])

In [5]:
print(supervisor.reply('Login to my file system'))

Task Completed!
Login Successfull


In [6]:
print(supervisor.reply('List files in my current directory'))

Transfering conversation to agent: "file_explorer_agent"
Here is the list of files in your current directory:

1. lmflow.png
2. .DS_Store
3. LICENSE
4. lmflow
5. output
6. rough
7. README.md
8. setup.py
9. .gitignore
10. examples
11. .ipynb_checkpoints
12. .git
13. .vscode


In [7]:
print(supervisor.reply('Does my current directory have a README file?'))

Yes, your current directory has a README file.


In [8]:
print(supervisor.reply('Change directory to output and list files')) # Does not handle sequence of tasks yet

Task Completed!
Changed to output


In [14]:
print(supervisor.reply('now list files'))

Here is the list of files in your current directory:

1. output_18-11-2024.csv
2. output_04-11-2024.csv
3. output_11-11-2024.csv
4. output_28-10-2024.csv
5. .ipynb_checkpoints


In [15]:
print(supervisor.reply("what is the size of output_28-10-2024.csv in this directory"))

Transfering conversation to agent: "file_operator_agent"
The size of output_28-10-2024.csv is 0 bytes.


In [16]:
print(supervisor.reply("delete file ending with 28-10-2024.csv"))

Please confirm the following action (Y/N): delete_file ('output_28-10-2024.csv',) {} n


Task Completed!
Task cancelled by user


In [17]:
print(supervisor.reply("What date is it?"))

Transfering conversation back to agent: "file_explorer_agent"
Transfering conversation back to agent: "supervisor"
Transfering conversation to agent: "timekeeper"
The date is 11-24-2024.


In [18]:
print(supervisor.reply("And what day is this?"))

Today is Sunday.
